In [ ]:
""" 
the input will be a string and based on this string i need to call a function 

option i am considering:

- use openai function calling
- create my own function calling framework

really good blog https://docs.mistral.ai/capabilities/function_calling/
"""

In [1]:
#tools 
def go_to_place_in_map(place):
    print(f"going to {place}")
    #thats it for now - later i will add more functionalit

In [2]:
tools = [
    {
        "type": "function",
        "function": {
            "name": "go_to_place_in_map",
            "description": "Go to a place in the map",
            "parameters": {
                "type": "object",
                "properties": {
                    "place": {
                        "type": "string",
                        "description": "The name of the place to navigate to.",
                    }
                },
                "required": ["place"],
            },
        },
    }
]

In [3]:
import functools

names_to_functions = {
    'go_to_place_in_map': functools.partial(go_to_place_in_map)
}

In [6]:
#this cell figures out what function to call
from openai import OpenAI
import os 

openai_api_key = os.environ.get("OPENAI_API_KEY")
client = OpenAI(api_key=openai_api_key)


system=f"""
You are a chatbot on top of a map. Your job is to help the user navigate the map. You can use the available functions to help you with this task.
"""
user_query="Go to berlin."
user_content=f"""
You are giving a chat conversation and based on this conversation you need to decide if an function from the available functions needs to be called. the available functions are:

---
{tools}
---

in the case a function can be called return the function name and the arguments that need to be passed to the function in JSON. Its important that the name of the key for the function name is `function_name` and the name of the key for the arguments is `arguments`. In the case a function cannot be called return an empty JSON object. 

Following the chat conversation:

---
{user_query}
---
"""



completion = client.chat.completions.create(
  model="gpt-3.5-turbo",
  response_format={ "type": "json_object" },
  temperature=0.0,
  messages=[
    {"role": "system", "content": system},
    {"role": "user", "content": user_content}
  ]
)
response_content = completion.choices[0].message.content

In [7]:
print(response_content)

{
  "function_name": "go_to_place_in_map",
  "arguments": {
    "place": "berlin"
  }
}


In [8]:
#this cell is executing a function if there is one to execute
import json

response_json = json.loads(response_content)

function_name = response_json["function_name"]
function_params = response_json["arguments"]
print(function_name, function_params)

function_result = names_to_functions[function_name](**function_params)

go_to_place_in_map {'place': 'berlin'}
going to berlin


In [11]:
#generate final answer
prompt_for_final_response="Provide a final response to the user"

completion = client.chat.completions.create(
  model="gpt-4-turbo",
  temperature=0.0,
  messages = [
    {"role": "system", "content": system},
    {"role": "user", "content": user_query},
    {"role": "assistant", "content": response_content},
    {"role": "user", "content": prompt_for_final_response}
]
)
response_content = completion.choices[0].message.content
print(response_content)

If you need any more information or further assistance while you're in Berlin, feel free to ask. Enjoy your time in the city!
